# Homework 09 Part 2
### Explanations are given below as we go.

##### I did not use the method given in the demo for the hammer problem. Because when I replicated the same method the reward given was always -1. Therefore the value for "scores" was always -200. This would result the car in never converging to the top. To combat the problem I had to change the method a little.

In [9]:
import gym
import random
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam

##### The number of goal steps is 200 because the time steps limit in the gym openai source code is 200. Initial games=10000 are the total number of iterations for the game to run.

In [10]:
env = gym.make('MountainCar-v0')
env.reset()
goal_steps = 200
score_requirement = -198
intial_games = 10000

##### Score, game_memory, previous_observation variables where will store the current game’s total score and previous step observation and the action taken.

##### The episode ends when you reach 0.5(top) position, or if 200 iterations are reached. I played several times 10000 times but never reached the top position. So at the time of data population, I changed a small logic that finally gave me the solution.

Then we will check whether the position of the car which is observation[0] is greater than -0.2 if yes then instead of taking the reward given by our game environment I took as 1 because -0.2 position is top of the hill which means our random actions giving somewhat fruitful results.
The agent will earn negative rewards for each action it takes making it very easy to land in the local optima of doing nothing. There are many approcahes implemented, this approach gave me the best results.

In [11]:
def model_data_preparation():
    training_data = []
    accepted_scores = []
    for game_index in range(intial_games):
        score = 0
        game_memory = []
        previous_observation = []
        for step_index in range(goal_steps):
            action = random.randrange(0, 3)
            observation, reward, done, info = env.step(action)
            
            if len(previous_observation) > 0:
                game_memory.append([previous_observation, action])
                
            previous_observation = observation
            if observation[0] > -0.2:
                reward = 1
            
            score += reward
            if done:
                break
            
        if score >= score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                if data[1] == 1:
                    output = [0, 1, 0]
                elif data[1] == 0:
                    output = [1, 0, 0]
                elif data[1] == 2:
                    output = [0, 0, 1]
                training_data.append([data[0], output])
        
        env.reset()
    
    print(accepted_scores)
    
    return training_data

In [12]:
training_data = model_data_preparation()

[-168.0, -180.0, -194.0, -186.0, -172.0, -192.0, -180.0, -198.0, -168.0, -182.0, -188.0, -188.0, -190.0, -192.0, -186.0, -198.0, -180.0, -174.0, -170.0, -170.0, -182.0, -194.0, -184.0, -194.0, -196.0, -170.0, -178.0, -192.0, -182.0, -174.0, -186.0, -194.0, -176.0, -184.0, -182.0, -184.0, -180.0, -170.0, -172.0, -168.0, -184.0, -182.0, -192.0, -192.0, -182.0, -180.0, -198.0]


In [13]:
def build_model(input_size, output_size):
        model = Sequential()
        model.add(Dense(128, input_dim=input_size, activation='relu'))
        model.add(Dense(52, activation='relu'))
        model.add(Dense(output_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam())

        return model

In [14]:
def train_model(training_data):
    X = np.array([i[0] for i in training_data]).reshape(-1, len(training_data[0][0]))
    y = np.array([i[1] for i in training_data]).reshape(-1, len(training_data[0][1]))
    model = build_model(input_size=len(X[0]), output_size=len(y[0]))
    
    model.fit(X, y, epochs=5)
    return model

In [15]:
trained_model = train_model(training_data)

Epoch 1/5
293/293 [==============================] - 1s 3ms/step - loss: 0.2294
Epoch 2/5
293/293 [==============================] - 1s 3ms/step - loss: 0.2221
Epoch 3/5
293/293 [==============================] - 1s 3ms/step - loss: 0.2210
Epoch 4/5
293/293 [==============================] - 1s 3ms/step - loss: 0.2204
Epoch 5/5
293/293 [==============================] - 1s 3ms/step - loss: 0.2202


In [16]:
scores = []
choices = []
for each_game in range(100):
    score = 0
    prev_obs = []
    for step_index in range(goal_steps):
        if len(prev_obs)==0:
            action = random.randrange(0,2)
        else:
            action = np.argmax(trained_model.predict(prev_obs.reshape(-1, len(prev_obs)))[0])
        
        choices.append(action)
        new_observation, reward, done, info = env.step(action)
        prev_obs = new_observation
        score+=reward
        if done:
            break

    env.reset()
    scores.append(score)

print(scores)
print('Average Score:',sum(scores)/len(scores))
print('choice 1:{}  choice 0:{} choice 2:{}'.format(choices.count(1)/len(choices),choices.count(0)/len(choices),choices.count(2)/len(choices)))

[-200.0, -126.0, -130.0, -127.0, -200.0, -128.0, -127.0, -128.0, -127.0, -200.0, -118.0, -117.0, -117.0, -126.0, -122.0, -132.0, -121.0, -200.0, -118.0, -127.0, -127.0, -119.0, -132.0, -117.0, -121.0, -200.0, -126.0, -200.0, -200.0, -200.0, -127.0, -126.0, -126.0, -121.0, -126.0, -120.0, -139.0, -118.0, -126.0, -122.0, -126.0, -119.0, -123.0, -126.0, -128.0, -127.0, -120.0, -200.0, -126.0, -127.0, -123.0, -118.0, -126.0, -125.0, -200.0, -123.0, -127.0, -131.0, -122.0, -127.0, -117.0, -131.0, -200.0, -200.0, -121.0, -120.0, -200.0, -128.0, -127.0, -200.0, -122.0, -125.0, -117.0, -127.0, -200.0, -136.0, -119.0, -119.0, -127.0, -200.0, -129.0, -117.0, -200.0, -119.0, -130.0, -127.0, -200.0, -129.0, -132.0, -123.0, -119.0, -200.0, -132.0, -119.0, -200.0, -126.0, -127.0, -132.0, -118.0, -123.0]
Average Score: -139.66
choice 1:0.008449090648718316  choice 0:0.21022483173421166 choice 2:0.7813260776170701


##### Our model trains knowing that if it goes to the top, it automatically performs well.

In [18]:
print ("Success rate: " +  str(abs(sum(scores)/goal_steps)))

Success rate: 69.83


##### I did not render the video as it was not explicitly mentioned in the homework description. Since colab is not locally hosted, env.render() does not produce any simulations of the gameplay. If you want to render the gameplay please run the below function and it will plot the frames at each action.

In [ ]:
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install pyvirtualdisplay
!pip install piglet
from pyvirtualdisplay import Display
Display().start()
import gym
from IPython import display
import matplotlib.pyplot as plt
%matplotlib inline
img = plt.imshow(env.render('rgb_array'))

scores = []
choices = []
for each_game in range(100):
    score = 0
    prev_obs = []
    for step_index in range(goal_steps):
        img.set_data(env.render('rgb_array'))
        display.display(plt.gcf())
        display.clear_output(wait=True)
        if len(prev_obs)==0:
            action = random.randrange(0,2)
        else:
            action = np.argmax(trained_model.predict(prev_obs.reshape(-1, len(prev_obs)))[0])
        
        choices.append(action)
        new_observation, reward, done, info = env.step(action)
        prev_obs = new_observation
        score+=reward
        if done:
            break

    env.reset()
    scores.append(score)

print(scores)
print('Average Score:',sum(scores)/len(scores))
print('choice 1:{}  choice 0:{} choice 2:{}'.format(choices.count(1)/len(choices),choices.count(0)/len(choices),choices.count(2)/len(choices)))

##### I ran the above simulation as well and my car moved up the mountain and successfully converged to the optimal solution rather than converging to the local minima.